# Convolution: Nuts & Bolts - I

In this notebook, we discuss the nuts and bolts of the convolution operation in the context of Convolutional Neural Networks (CNNs) or convnets. We try to understand how convnets use the convolution operation to solve perception tasks such as vision. 

Convnets are designed based on the architecture of visual cortex. The **visual world is fundamentally spatially hierarchical**. Thus, to percieve an input image convnets create successive layers of representations of the image. Convnets employ convolutional filters (also known as kernels) that scan across the image for detecting local features, thus create feature maps. These feature maps are then scanned by filters in the successive layers.

The first convolution layer learns small local patterns such as edges, the second convolution layer learns larger patterns made of the features of the first layers, and so on. This allows convnets to efficiently learn increasingly complex and abstract visual concepts. The final layer of a convnet uses high-level representations to extract class-level information (e.g., whether the image represents a cat), as shown below.

<img src="https://cse.unl.edu/~hasan/Pics/CNN_Spatial_Hierarchy_Visual_Modules.png" width=600, height=300>


The fundamental technique for detecting local features (from the input image or the subsequent feature maps) is the convolution operation. But what is convolution?


## Convolution: Intuition

Let's have an intuitive understanding of convolution. Consider the following image.

<img src="https://cse.unl.edu/~hasan/Pics/CNN_Image_Original.png" width=300, height=100>

The image shows the face of the person in granular detail. Say that we want to squash the details and create a smoother version of it as follows.

<img src="https://cse.unl.edu/~hasan/Pics/CNN_Image_Smoothed.png" width=300, height=100>


How do we do this smoothing operation? 

We can use a simple approach. Replace each pixel by the average value of its surrounding pixels. For example, we can take all pixels within one-pixel neighborhood of a given pixel, compute their mean, and replace the given pixel by the mean. Then, we get a new image with somoothed out pixel values.

To implement the above approach, we can use a fixed-size window to slide over all regions of the image and compute the mean of the overlap region. 

As an illustration of this implementation, let's say that we have a 4 x 4 image (on the left below) that we want to smooth out. The pixel values for each cell are shown. We slide a 3 x 3 window from left to right and top to bottom. At each step of the slide, the 3 x 3 window is set on a cell (e.g., the top-left cell) that we want to smooth out. Then, the 9 values in the overlap region are averaged (i.e., 2.25). This new value replaces the old pixel value (e.g., top-left pixel on the right). This way we create a smoothed out version of the original "image" on the right.

<img src="https://cse.unl.edu/~hasan/Pics/CNN_Smoothing.png" width=600, height=400>


This operation of sliding a fixed-size window over the image and to compute new values for the overlap region resembles the convolution operation. The sliding window is known as convolution filter or kernel. The output is known as feature map. 


## Convolution: in Convnets

Convnets use the convolution operation (very similar to the above-mentioned smoothing operation) to detect local features. What exatly are these local features?

Consider the following image.

<img src="https://cse.unl.edu/~hasan/Pics/CNN_Image_Bench.png" width=300, height=100>

This image consists of a set of distinctive vertical and horizontal lines. For perceiving this image, first, we need to detect the vertical and horizontal "edges", which are low-level features. Then, these features are assembled to construct the complete perception of the image. 

A convnets starts with detecting the low-level features (i.e., edges) via the convolution operation. Following figure shows the output (feature map) created by two convolutional filters: one for vertical edge detection and other for horizontal edge detection.

<img src="https://cse.unl.edu/~hasan/Pics/CNN_Image_Bench_Edges.png" width=600, height=400>

Thus, convolution operation is a fundamental technique in convnets to detect local features of an image. But how do we design these filters?

We actually don't! In a convnet we only create an architecture for these filters (size, number, layer). A convnet **learns** these filters via **Backpropagation** algorithm for detecting features suitable for the target task (e.g., classification).

In this notebook we don't discuss the learning technique. Instead we handcraft some filters to uderstand the process of convolution.



## Tasks

Since convolution is the most fundamental operation in convnets, we try to understand it thoroughly. We discuss the following aspects of convolution.

- Convolution: Mathematical Understanding
- Convolution: Mathematical Formula for Single-Channel 2D Input & Single 2D Filter
- Convolution as Cross-Correlation
- Convolution: Multi-Channel 2D Input & Multiple 2D Filters
- Convolution with Varying Strides
- Convolution: Mathematical Formula for Multi-Channel 2D Input & Multiple 2D Filters
- Convolution: Python Implementation

## Convolution: Mathematical Understanding



Convolution is a special type of **linear operation**. In general, it's an operation on two functions of a real-valued argument. We re-use the smoothing example to present a mathematical definition of convolution. 

## Convolution: Continuous Input Function - 1D Case

We simplify the problem by considering a one-dimensional function. Say that we want to smooth out a one-dimensional function x(t). Here "t" denotes the location in 1D space.

To smooth x(t), we replace each point on x(t) by the average value of its neighborhood. When we do this averaging, we want to give more weight to the nearest values. In other words, we want to compute a **weighted average** that gives more weight to nearest values. We can do this by using a weighting function w(). 

If we apply such a weighted average operation at the current location of the function x() (by multiplying x with w), we obtain a new function z() that provides a smoothed estimate of x().

Mathematically speaking, we want to get the weighted average of x() at the current location "i". This average is denoted by z(i). We look at the neighborhood of x from $-\infty$ to $+\infty$. We denote the range with "t".

$z(i) = \int_{t = -\infty}^{+\infty} x(t)w(i - t)dt$

This **weighted averaging operation operation is called convolution**. The convolution operation is typically denoted with an asterisk.

$z(t) = (x ∗ w)(t)$

Note that in this example, "w" needs to be a valid probability density function, or the output is not a weighted average.

In the convnet terminology, the first argument (the function x) to the convolution is often referred to as the input and the second argument (the function w) as the kernel or filter. The output is referred to as the feature map.

The above example of convolution is based on a continuous function of continuously varying position "t". In convnets, positions are discretized. Thus, we are interested about discrete convolution.


## Discrete Convolution: 1D Case

In discrete convolution, the position index "t" can take on only integer values. If we now assume that x and w are defined only on integer t, we can define the discrete convolution as follows.

$z(i) = \sum_{t = -\infty}^{+\infty} x(t)w(i - t)$


In convnets, the input is usually a multidimensional array of data (e.g., 3D RGB image) and the kernel is usually a multidimensional array of parameters that are adapted by the learning algorithm. These multidimensional arrays are typically represented by tensors.


## Multidimensional Convolution

Note that to implement convolution in covnets, we need to explicitly store each element of the input and kernel separately. Thus, we usually assume that these functions are zero everywhere except for the finite set of points for which we store the values. This means that in practice we can implement the infinite summation as a summation over a finite number of array elements.

To define the equation for multidimensional convolution, we first consider the 2D case. More specifically, we use a single-channel 2D image as input and a single 2D kernel as filter, which produces a 2D feature map. After understanding the 2D case, we will define the convolution equation for multi-channel and multi-filter case.


## Convolution: Single-Channel 2D Input & Single 2D Filter

Consider a single-channel 2D image $X$ where $X(a, b)$ is the pixel intensity at x-y coordinates denoted by a,b. Since the input has two dimensions, we want to apply convolution over these two axes at a time. Thus, we need to use a single 2D kernel $K$. We get a new image (feature map) denoted by $Z$ by performing 2D convolution as follows.

$Z(i, j) = \sum_{a} \sum_{b} X(a, b) * K(i - a, j - b)$


## Convolution: An Alternative (Widely Used) Implementation

So far, we have defined the convolution operation mathematically for the 2D scenario. We now want to add a twist in the implementation of convolution.

It appears that if we try to implement convolution according to the above definition, then we will have to **flip the kernel matrix**. Many neural network libraries don't implement the kernel-flipping version of convolution. Instead they use a straight-forward technique. Their technique is known as **cross-correlation**, which is the same as convolution but without flipping the kernel.

When we code the convolution operation for a convnet, we find it conveient to implement the cross-correlation operation. Below we present cross-correlation.


## Convolution as Cross-Correlation

Before we define the cross-correlation function, let's understand why do we have to flip the kernel in the original convolution operation.

Consider the following 2D kernel.

<img src="https://cse.unl.edu/~hasan/Pics/CNN_2DFilter_Flipped.png" width=200, height=100>

This 3 x 3 kernel is center originated. It means that the center point of the kernel is K[0, 0]. For this 3 x 3 kernel the array index of 3 elements will be -1, 0, and 1. 

Now, let’s consider a 3 x 3 region of a 2D input image denoted by $X$. The 2D coordinates of each location (e.g., pixel) of the 3 x 3 region are shown in the figure.

<img src="https://cse.unl.edu/~hasan/Pics/CNN_2DInput.png" width=400, height=200>

Let’s perform the convolution at the location (1, 1) of the feature map $Z$.

$Z(i, j) = \sum_{a=0}^{2} \sum_{b=0}^{2} X(a, b) * K(i - a, j - b)$

=> $Z(1,1)=X(0,0)K(1,1) + X(0,1)K(1,0) + X(0,2)K(1,-1) + X(1,0)K(0,1) + X(1,1)K(0,0) + X(1,2)K(0,-1) + X(2,0)K(-1,1)  + X(2,01)K(-1,0) + X(2,2)K(-1,-1)$

This calculation is illustrated below.

<img src="https://cse.unl.edu/~hasan/Pics/CNN_Convolution_FlippedFilter.png" width=600, height=400>

Notice that before multiplying with the input image, we had to **flip the kernel matrix** in both horizontal and vertical directions. This is because X[0,0] is multiplied by the last element of the kernel K[1,1]. And X[2,2] is multiplied by the first element of the kernel K[-1,-1].

- Can we do convoltuion without flipping the kernel?

We may want to perform the convolution like this. 

<img src="https://cse.unl.edu/~hasan/Pics/CNN_Convolution_Cross-Correlation.png" width=600, height=400>

But how do we define the above operation mathematically?

We do this type of convolution without flipping the kernel by using cross-correlation, which is defined as follows.

$Z(i, j) = \sum_{a} \sum_{b} X(i + a, j+ b) * K(a, b)$


Observe that now a shift in the input $X$ leads to a shift in the feature map $Z$. This was possible because now $K$ does not depend on $(i, j)$.

Using the cross-correlation function, let’s perform convolution for the previous example, i.e., at the location (1, 1) of $Z$.

$Z(i, j) = \sum_{a=-1}^{1} \sum_{b=-1}^{1} X(i + a, j + b) * K(a, b)$

=> $Z(1,1)=X(0,0)K(-1,-1) + X(0,1)K(-1,0) + X(0,2)K(-1,1) + X(1,0)K(0,-1) + X(1,1)K(0,0) + X(1,2)K(0,1) + X(2,0)K(1,-1) + X(2,01)K(1,0) + X(2,2)K(1,1)$


That's it! We have defined the practical equation of convolution for the 2D case (both input and filter) as cross-correlation.

- We will refer to the cross-correlation operation as convolution to conform with the convention in Deep Learning literature.

# Convolution: Multi-Channel 2D Input & Multiple Filters

We will define the mathematical equation for performing convolution using multi-channel 2D input and muliple filters to produce multi-channel output feature maps. To do this, first, we develop a visual understanding of this process. We begin by revisiting the single-channel 2D input and single 2D filter case.


## Visual Tour: Convolution - Single-Channel 2D Input & Single 2D Filter
Consider the following figure that shows a 5 x 5 input image (denoted by X) and a 3 x 3 convolutional filter or kernel (denoted by K). The image is grayscale, i.e., it has only one channel. Our intention is to create a single feature map, that's why we have only one filter that is single-channel. 


<img src="https://cse.unl.edu/~hasan/Pics/CNN_Image_Filter.png" width=400, height=200>

Here both the single-channel input and the single-channel filter are represented by 2D matrices. To perform convolution, we slide the filter over all regions of the image, move it by one element (pixel) from top to bottom, left to right. For each overlap region, we compute an element-wise multiplication of these two 2D matrices (the input and the filter), followed by a sum, as shown below. The convolution operation creates a 3 x 3 feature map.

<img src="https://cse.unl.edu/~hasan/Pics/CNN_Convolution.png" width=800, height=600>


## Convolution with Varying Strides

Before we discuss the multi-channel 2D Input & multi-channel filter scenario, let's discuss how the size of the output feature map varies based on our sliding strategy.

When performing convolution, we start with the convolution window at the top-left corner of the input tensor, and then slide it over all locations both down and to the right. By default, we slide or "stride" one input pixel at a time. This process is known as convolution with a stride of length 1, as shown below.

<img src="https://cse.unl.edu/~hasan/Pics/Padding_VALID_Stride1.png" width=700, height=400>


However, sometimes, we need to stride (slide) the filter by more than one pixel at a time (i.e., stride > 1), skipping the intermediate locations.

Following figure shows the impact of using a stride of length 2. We see that the feature map gets reduced to 2 x 2.

<img src="https://cse.unl.edu/~hasan/Pics/CNN_Convolution_Stride2.png" width=600, height=400>

A stride > 1 is used: 
- To achieve computational efficiency or
- To downsample.


## Size of the Feature Map

In general, the convolution operation reduces the size of an image (feature map). The shape of the output feature map is determined by the shape of the input and the shape of the convolution filter (or kernel). A single-channel input $X$ (image or feature map) of height $n_h$ and width $n_w$, when convolved with a single-channel filter $K$ of height $f_h$ and width $f_w$ using a stride $s$, reduces the size of its output (feature map) $Z$ by:

- Z (height): $(n_h - f_h)/s + 1$
- Z (width): $(n_w - f_w)/s + 1$



## Visual Tour: Convolution - Multi-Channel 2D Input & Multi-Channel Filter

Often times input is multi-channel (e.g., RGB image contains three channels Red, Green and Blue). We represent a multi-channel input using a 3D matrix, or more conveniently using a 3D tensor (height, width, number of channels). When the input data contains multiple channels, we need to construct a convolution kernel with the same number of input channels as the input data, so that it can perform convolution with the input data. 

Thus, for the input RGB image a single filter with 3 channels is used to convolve this 3D input. The single multi-channel filter is represented by a 3D tensor as well (height, width, number of input channels).


<img src="https://cse.unl.edu/~hasan/Pics/CNN_RGB_Input_SingleFilter.png" width=500, height=300>

For the multiple channels of the input, each channel of the filter scans through the same region on all input channels, compute element-wise multiplication followed by sum, as shown below. Note that the single multi-channel filter produces a single feature map.

<img src="https://cse.unl.edu/~hasan/Pics/CNN_RGB_Convolution_SingleFilter.png" width=600, height=300>


## Visual Tour: Convolution - Multi-Channel 2D Input & Multiple 3D Filters


So far we have seen that irrespective of the number of input channels, we always get one feature map, i.e., single-channel output. However, in practice it is essential to create multiple feature maps. In most convnet architectures, the channel dimension gets smaller as the network gets deeper. But to compensate the loss in spatial resolution, number of output channels are increased. Each output channel or feature map detects different set of features. The feature maps don't learn representaions independently, instead they work in a coalition. For example, it may be the case that a single channel featue map does not by itself detect edges, instead some direction in channel space corresponds to detecting edges.


For creating multiple feature maps or multi-channel output from a multi-channel input, we use multiple 3D filters. This filter set is stacked together and is represented by a 4D tensor (height, width, number of input channels, number of filters).

<img src="https://cse.unl.edu/~hasan/Pics/CNN_RGB_Input_MultipleFilters.png" width=500, height=300>

It is customary to show the multi-channel input, multiple 3D filters and feature maps by using 3D cubes.

<img src="https://cse.unl.edu/~hasan/Pics/CNN_RGB_Cube.png" width=700, height=400>




## Formalization: Convolution -  Multi-Channel 2D Input & Multiple 3D Filters

Let's formalize the convolution operation for multi-channel 2D input using multiple 3D filters. Consider the following figure. We have $n_k$ 2D input channels (or feature maps) in layer $l - 1$. This input is represented by a 3D tensor of size $n_h \times n_w \times n_k$. There are a set of $f_k$ 3D convolutional filters in layer $l$. The size of each filter is $f_h \times f_w \times n_k$, which is represented by a 3D tensor. The set of $f_k$ 3D filters is a 4D tensor of size $f_h \times f_w \times n_k \times f_k$.

<img src="https://cse.unl.edu/~hasan/Pics/CNN_MultiChannelInput_MultipleFilters_1.png" width=700, height=400>

The $f_k$ filters produce $f_k$ 2D feature maps in layer $l$.

<img src="https://cse.unl.edu/~hasan/Pics/CNN_MultiChannelInput_MultipleFilters_2.png" width=700, height=400>

Let's consider a single 3D filter. Each of its $n_k$ channels separately convolve with the $n_k$ channels of input to produce a single feature map. In this example, the input height and width are 5 x 5. It is convolved with 3 x 3 filters with stride 2. Thus, the size of the output feature map is 2 x 2.

<img src="https://cse.unl.edu/~hasan/Pics/CNN_MultiChannelInput_MultipleFilters_3.png" width=700, height=400>




## Convolution: Mathematical Formula for Multi-Channel 2D Input & Multiple 3D Filters

Based on the visual understanding of the process of convolution and varying strides, we are ready to define the mathematical formula for performing convolution for multi-channel 2D input using multiple 3D filters.



Consider the following figure. We will show how to compute the value of a single cell or neuron of a single feature map $Z(i, j, k)$. Here $i$ and $j$ denote the row and column locations of the feature map, respectively; and $k$ denotes the $k$-th feature map.




<img src="https://cse.unl.edu/~hasan/Pics/CNN_Convolution_Equation_1.png" width=700, height=400>



The neuron at $Z(i, j, k)$ is connected to the outputs of the neurons of **a single 3D filter** in the previous layer $l-1$ (the neurons in the input layer are represented by the tensor $X$). In the above illustration we show the convolution of a single input channel (denoted by $k_{in}$ by a single channel of the filter). The locations of the scanned neurons at the input layer $l - 1$ are indexed as follows:

- Row: $i \times s_h$ to $i \times s_h + f_h – 1$
- Column: $j \times s_w$ to $j \times s_w + f_w – 1$


Following figure illustrates how this indexing works when sliding a 3 x 3 filter on a 6 x 6 input with stride 1.

<img src="https://cse.unl.edu/~hasan/Pics/CNN_Convolution_Equation_2a.png" width=800, height=700>


Carefully note that on the feature map all neurons located in the same row $i$ and column $j$ but in different feature maps are connected to the outputs of the exact same neurons in the previous layer.

Based on the above illustration we provide the equation for computing the output of a neuron in a convolutional layer.

$Z(i, j, k) = b_k + \sum_{a=0}^{f_h - 1}\sum_{b=0}^{f_w - 1}\sum_{k_{in}=0}^{n_k}X(i\times s_h + a,
j\times s_w + b, k_{in}) * K(a, b, k_{in}, k)$

Here $b_k$ is the bias neuron. Each feature map has one bias neuron.

Following figure shows the computation of the value of a neuron of a single feature map $Z(0, 0, 0)$.

<img src="https://cse.unl.edu/~hasan/Pics/CNN_Convolution_Equation_3.png" width=700, height=400>


## Convolution: Python Implementation

The above definition of convolution implements the **cross-correlation** operation. In convnets, a convolutional layer cross-correlates the input and filter (kernel) and adds a scalar bias to produce an output (feature map). We will refer to the cross-correlation operation as convolution to conform with the convention in Deep Learning literature.

We define the following convolution (which is actually cross-correlation) function to perform 2D convolution on a single channel input by a single-channel 2D filter. Later we will provide implementation for the multi-channel 2D input and multiple 3D filters case. 

The function takes a 2D input (X), 2D filter (K), and a scalar stride (s).

In [1]:
import numpy as np

def conv_2D_SingleChannelInput_SingleFilter(X, K, s): 
    # Get the height and width of the filter
    f_h, f_w = K.shape
    
    # Get the height and width of the input
    n_h, n_w = X.shape
    
    # Create the output (feature map) matrix
    Z = np.zeros(((n_h - f_h)//s + 1, (n_w - f_w)//s + 1))
    
    # Perform cross-correlation
    for i in range(Z.shape[0]):
        for j in range(Z.shape[1]):
            element_wise_product = X[i*s: i*s + f_h, j*s: j*s + f_w] * K
            Z[i, j] = np.sum(element_wise_product)          
    return Z

## Convolution: Illustration

We perform convolution on a single channel 2D input matrix using:
- Stride = 1
- Stride = 2

In [2]:
X = np.array([[1, 1, 1, 0, 1], 
              [0, 0, 0, 1, 0], 
              [1, 1, 1, 0, 1], 
              [0, 0, 0, 1, 0], 
              [1, 1, 1, 0, 1]])
print("Input (X):\n", X)

K = np.array([[1, 0, 1], 
              [0, 1, 0], 
              [1, 0, 1]])
print("\nFilter (K):\n", K)

print("___________________________________\n")

print("Convolution (stride 1): Feature Map Z")
Z_1 = conv_2D_SingleChannelInput_SingleFilter(X, K, 1)
print(Z_1)

print("\nConvolution (stride 2): Feature Map Z")
Z_2 = conv_2D_SingleChannelInput_SingleFilter(X, K, 2)
print(Z_2)

Input (X):
 [[1 1 1 0 1]
 [0 0 0 1 0]
 [1 1 1 0 1]
 [0 0 0 1 0]
 [1 1 1 0 1]]

Filter (K):
 [[1 0 1]
 [0 1 0]
 [1 0 1]]
___________________________________

Convolution (stride 1): Feature Map Z
[[4. 2. 5.]
 [1. 3. 0.]
 [4. 2. 5.]]

Convolution (stride 2): Feature Map Z
[[4. 5.]
 [4. 5.]]


## Next ...

In the next notebook.
- We will perform edge detection using the above convolution function.
- We will define the convolution function for multi-channel input using multiple filters.